In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import entropy
import datetime
import copy
import time
import wandb
import pickle as pkl
import sys
sys.path.append('../../')
from data.processing import get_data
import models
from main.seir.fitting import single_fitting_cycle
from main.seir.forecast import get_forecast, forecast_all_trials, create_all_trials_csv, create_decile_csv_new
from main.seir.sensitivity import calculate_sensitivity_and_plot
from utils.generic.create_report import save_dict_and_create_report
from utils.generic.config import read_config
from utils.generic.enums import Columns
from utils.fitting.loss import Loss_Calculator
from utils.generic.logging import log_wandb
from viz import plot_forecast, plot_top_k_trials, plot_ptiles
from viz.fit import plot_histogram, plot_all_histograms, plot_mean_variance, plot_scatter, plot_kl_divergence, plot_heatmap_distribution_sigmas, plot_all_params, plot_all_losses, plot_all_buckets, plot_cv_in_params, plot_recovery_loss
import yaml
from data.dataloader import SimulatedDataLoader

In [ ]:
predictions_dict = {}

In [ ]:
output_folder = '../../misc/reports/{}'.format(datetime.datetime.now().strftime("%Y_%m%d_%H%M%S"))

In [ ]:
predictions_dict.keys()

In [ ]:
num_rep_trials = 2
actual_params = []
simulated_data_configs = ['seir_pu_variable.yaml']
for config_filename in simulated_data_configs:
    for m in range(num_rep_trials):
        with open(os.path.join("../../configs/simulated_data/", config_filename)) as configfile:
            config = yaml.load(configfile, Loader=yaml.SafeLoader)    
        config['output_file_name'] = config_filename.split('.')[0] + '_' + str(m) + '.csv'
        loader = SimulatedDataLoader()
        _, actual_param = loader.load_data(**config)
        actual_params.append(actual_param)

In [ ]:
config_filenames = ['experiments/seirhd.yaml']
model_params = {
        'SEIRHD': [ 'lockdown_R0', 'T_inc', 'T_inf', 'T_inf', 'T_recov', 'T_recov_fatal', 'P_fatal', 'E_hosp_ratio', 'I_hosp_ratio'],
    }
model_types = {'SEIRHD' : 'SEIRHD', 'SEIRHD Free' : 'SEIRHD', 'SEIRHD Cons' : 'SEIRHD', 'SEIR_Undetected':'SEIR_Undetected','SEIR_PU':'SEIR_PU','SEIR_PU_Testing':'SEIR_PU_Testing'}
model_names = list(model_params.keys())
configs = [read_config(config_filename) for config_filename in config_filenames]
# param_tuples = {
#     'free':{},
#     'lat_params_fixed':{'I_hosp_ratio': 0.5, 'E_hosp_ratio': 0.5,'Pu_pop_ratio': 0.3}
# }
param_tuples = {
    # 'all':{'I_hosp_ratio': 0.5, 'E_hosp_ratio': 0.5,'lockdown_R0': 0.8,'P_fatal':0.3,'T_recov_fatal':25,'T_recov':15,'T_inc':4.5,'T_inf':5}
    'P_fatal':{'I_hosp_ratio': 0.5, 'E_hosp_ratio': 0.5,'lockdown_R0': 0.8,'T_recov_fatal':25,'T_recov':15,'T_inc':4.5,'T_inf':5},
    'r0':{'I_hosp_ratio': 0.5, 'E_hosp_ratio': 0.5,'P_fatal':0.3,'T_recov_fatal':25,'T_recov':15,'T_inc':4.5,'T_inf':5},
    'T_inf':{'I_hosp_ratio': 0.5, 'E_hosp_ratio': 0.5,'lockdown_R0': 0.8,'P_fatal':0.3,'T_recov_fatal':25,'T_recov':15,'T_inc':4.5}
    # 'free':{}
}

In [ ]:
configs = [read_config(config_filename) for config_filename in config_filenames]
num_rep_trials = 2
for tag, loc in param_tuples.items():
    predictions_dict[tag] = {}
    for j, config in enumerate(configs):
        predictions_dict[tag][model_names[j]] = {}
        config_params = copy.deepcopy(config['fitting'])
        for param in loc:
            if param in config_params['variable_param_ranges']:
                del config_params['variable_param_ranges'][param] 
            config_params['default_params'][param] = loc[param]
        print ('variable param ranges:', config_params['variable_param_ranges'])
        print ('default param ranges:', config_params['default_params'])
        for k in range(num_rep_trials):
            predictions_dict[tag][model_names[j]][f'm{k}'] = single_fitting_cycle(**config_params) 

In [ ]:
save_dir = '../../misc/predictions/deg_exp/'    
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
for tag, tag_dict in predictions_dict.items():
    with open(os.path.join(save_dir, tag + ".pickle"), 'wb') as handle:
        pkl.dump(tag_dict, handle)

### Use the pickle file to read the predicitons_dict

In [ ]:
save_dir = '../../misc/predictions/deg_exp/'    
predictions_dict_complete = {}
files_to_read = []
files = [i+'.pickle' for i in files_to_read if i+'.pickle' in os.listdir(save_dir)]
if len(files) == 0:
    files = os.listdir(save_dir)
for file_name in files:
    with open(os.path.join(save_dir, file_name), 'rb') as handle:
        predictions_dict_complete[file_name.split('.')[0]] = pkl.load(handle)

In [ ]:
# wandb.init(project="covid-modelling")
# wandb.run.name = "degeneracy-exps-location"+wandb.run.name

In [ ]:
plot_all_params(predictions_dict_complete, model_params, method='ensemble_combined')

In [ ]:
with open(os.path.join("../../configs/simulated_data/", 'seir_pu_fixed.yaml')) as configfile:
    config_simlated_data = yaml.load(configfile, Loader=yaml.SafeLoader) 
actual_param = config_simlated_data['params']
plot_recovery_loss(predictions_dict_complete, actual_param, model_params, method='ensemble_combined')

In [ ]:
# coefficient of varience
plot_cv_in_params(predictions_dict_complete, model_params)

In [ ]:
which_compartments = {model_names[i]: config['fitting']['loss']['loss_compartments'] for i, config in enumerate(configs)}
plot_all_losses(predictions_dict_complete, which_losses=['train', 'val'], which_compartments=which_compartments,method='best_loss_nora')
which_compartments = {model_names[i]: [] for i, config in enumerate(configs)}
# plot_all_losses(predictions_dict_complete, which_losses=['train'], which_compartments=which_compartments,method='best_loss_ra')
# plot_all_losses(predictions_dict_complete, which_losses=['train'], which_compartments=which_compartments,method='ensemble_loss_ra')

In [ ]:
plot_all_buckets(predictions_dict, which_buckets=['S', 'I', 'E', 'I_U'], compare='model', model_types=model_types)